### Add sentiments to, clean up and engineer the twitter data and combine it with stock data
Use this notebook for adding sentiments, cleaning and engineering the twitter data and combining it with stock data (data scraped during March 2019)

Resources:
Original twitter data from 2016 for the training:
http://followthehashtag.com/datasets/nasdaq-100-companies-free-twitter-dataset/

Sentiment analysis:
https://github.com/cjhutto/vaderSentiment

In [1]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import datetime as dt
import pandas_datareader.data as web
import math
import xlsxwriter
import os

### Change file- and stockname/cashtag and run all cells
In the cell below, as well as the last cell in this notebook, for each new stock change file- and stockname/cashtag and run all cells.

In [390]:
xls = pd.ExcelFile('C:/Users/jangn/OneDrive/CODE/Stock_markets/Cashtag_Tweets_scraper/$WDC/$WDC_3.xlsx') # CHANGE FILE NAME!!!

stock = "WDC" #CHANGE STOCK TICKER!!!

In [391]:
df = pd.read_excel(xls, header = 0,encoding='latin-1')

df.head()

,created at,text,follower count
0,2019-03-28 18:43:06,Bullish Option Order Flow Sentiment March 28 h...,2089
1,2019-03-28 18:10:26,RT @247WallSt: Thursday’s Top Analyst Upgrades...,32
2,2019-03-28 17:55:39,$WDC in at $45.68,49
3,2019-03-28 17:54:13,$WDC potential EOD runner,49
4,2019-03-28 17:24:12,Western Digital $WDC Holding Boosted by Orca I...,419


### Add column with just the date, remove column with date & time, rearrange columns

In [392]:
df["date"] = df["created at"].dt.date
df.tail()

,created at,text,follower count,date
1113,2019-02-28 23:13:26,Thurs (2/28/19) highest % above OR 30-min high...,792,2019-02-28
1114,2019-02-28 22:55:24,RT @SharePlanner: Stocks For You To Swing-Trad...,186,2019-02-28
1115,2019-02-28 22:44:00,RT @SharePlanner: Stocks For You To Swing-Trad...,149,2019-02-28
1116,2019-02-28 22:28:30,"$MNST gapped up on #earnings, gaining over 8% ...",1569,2019-02-28
1117,2019-02-28 21:00:02,Here is a snapshot of the current market. Reme...,1292,2019-02-28


### Add sentiment to each tweet using Vader
Extract the content.<br>
Make sure all tweets with an empty text field are deleted before doing the senitment analysis!

In [393]:
Tweet = df['text']
Tweet.head()

0    Bullish Option Order Flow Sentiment March 28 h...
1    RT @247WallSt: Thursday’s Top Analyst Upgrades...
2                                    $WDC in at $45.68
3                            $WDC potential EOD runner
4    Western Digital $WDC Holding Boosted by Orca I...
Name: text, dtype: object

In [394]:
def sentimentScore(Tweet):
    analyzer = SentimentIntensityAnalyzer()
    results = []
    for sentence in Tweet:
        vs = analyzer.polarity_scores(sentence)
        print("Vader score: " + str(vs))
        #print("{: <65} {}".format(sentence, str(vs))) 
        #NOTE! I blocked the second print command so the sentences are
        #left out in the cell below, purely for clarity reasons
        results.append(vs)
    return results

In [ ]:
df_results = pd.DataFrame(sentimentScore(Tweet));

### Combining the two dataframes
Next, I combine the **df_tweets** dataframe with the **df_results** sentiment dataframe

In [396]:
df_results.head()

,compound,neg,neu,pos
0,0.0000,0.000,1.000,0.000
1,0.2023,0.000,0.917,0.083
2,0.0000,0.000,1.000,0.000
3,0.0000,0.000,1.000,0.000
4,0.4939,0.048,0.801,0.151


In [397]:
df_tweets = pd.merge(df, df_results, left_index=True, right_index=True)

df_tweets.tail()

,created at,text,follower count,date,compound,neg,neu,pos
1113,2019-02-28 23:13:26,Thurs (2/28/19) highest % above OR 30-min high...,792,2019-02-28,0.4404,0.032,0.896,0.072
1114,2019-02-28 22:55:24,RT @SharePlanner: Stocks For You To Swing-Trad...,186,2019-02-28,0.0000,0.000,1.000,0.000
1115,2019-02-28 22:44:00,RT @SharePlanner: Stocks For You To Swing-Trad...,149,2019-02-28,0.0000,0.000,1.000,0.000
1116,2019-02-28 22:28:30,"$MNST gapped up on #earnings, gaining over 8% ...",1569,2019-02-28,0.7351,0.000,0.829,0.171
1117,2019-02-28 21:00:02,Here is a snapshot of the current market. Reme...,1292,2019-02-28,0.3595,0.000,0.935,0.065


### Converting 'date' column from object to datetime

In [398]:
df_tweets['date'] = pd.to_datetime(df_tweets['date'])

### Choose the common range for the dataframes to be used for all tweet data
Using range 28.2.2019 - 28.3.2019 = nn days

In [399]:
df_tweets = df_tweets[(df_tweets['date'] >= '2019-02-28') & (df_tweets['date'] <= '2019-03-28')]

df_tweets.tail()

,created at,text,follower count,date,compound,neg,neu,pos
1113,2019-02-28 23:13:26,Thurs (2/28/19) highest % above OR 30-min high...,792,2019-02-28,0.4404,0.032,0.896,0.072
1114,2019-02-28 22:55:24,RT @SharePlanner: Stocks For You To Swing-Trad...,186,2019-02-28,0.0000,0.000,1.000,0.000
1115,2019-02-28 22:44:00,RT @SharePlanner: Stocks For You To Swing-Trad...,149,2019-02-28,0.0000,0.000,1.000,0.000
1116,2019-02-28 22:28:30,"$MNST gapped up on #earnings, gaining over 8% ...",1569,2019-02-28,0.7351,0.000,0.829,0.171
1117,2019-02-28 21:00:02,Here is a snapshot of the current market. Reme...,1292,2019-02-28,0.3595,0.000,0.935,0.065


### Adding a datetime column/index
Next, removing the 'Date' column and adding a datetime column/index. This format is needed for the daily average calculations!

In [400]:
df_tweets['datetime'] = pd.to_datetime(df_tweets['date']) # change of created at column to datetime columns
df_tweet_SA = df_tweets.set_index('datetime') # creates a new dataframe 'df_tweet_SA' with the new index column datetime 
df_tweet_SA.drop(['date'], axis=1, inplace=True) #drops the original 'created at' column from the dataframe
df_tweet_SA.tail()

,created at,text,follower count,compound,neg,neu,pos
datetime,,,,,,,
2019-02-28,2019-02-28 23:13:26,Thurs (2/28/19) highest % above OR 30-min high...,792,0.4404,0.032,0.896,0.072
2019-02-28,2019-02-28 22:55:24,RT @SharePlanner: Stocks For You To Swing-Trad...,186,0.0000,0.000,1.000,0.000
2019-02-28,2019-02-28 22:44:00,RT @SharePlanner: Stocks For You To Swing-Trad...,149,0.0000,0.000,1.000,0.000
2019-02-28,2019-02-28 22:28:30,"$MNST gapped up on #earnings, gaining over 8% ...",1569,0.7351,0.000,0.829,0.171
2019-02-28,2019-02-28 21:00:02,Here is a snapshot of the current market. Reme...,1292,0.3595,0.000,0.935,0.065


### Slimming down the stream into a dataframe with only relevant columns.

In [401]:
df_tweet_SA = df_tweets[['datetime', 'text', 'follower count', 'compound','neg','neu','pos']]

df_tweet_SA.head()

,datetime,text,follower count,compound,neg,neu,pos
0,2019-03-28,Bullish Option Order Flow Sentiment March 28 h...,2089,0.0000,0.000,1.000,0.000
1,2019-03-28,RT @247WallSt: Thursday’s Top Analyst Upgrades...,32,0.2023,0.000,0.917,0.083
2,2019-03-28,$WDC in at $45.68,49,0.0000,0.000,1.000,0.000
3,2019-03-28,$WDC potential EOD runner,49,0.0000,0.000,1.000,0.000
4,2019-03-28,Western Digital $WDC Holding Boosted by Orca I...,419,0.4939,0.048,0.801,0.151


### Remove tweets were compound is zero, i.e. sentiment is neutral
Neutral tweets do not reflect a positive or negative mood and serve therefore no purpose to this analysis. 

In [402]:
df_tweet_SA = df_tweet_SA[(df_tweet_SA[['compound']] != 0).all(axis=1)]

### Create new column with the 'compound' multiplied by nr of followers of the Tweeter
The logic being, if the tweeter has many followers, the compound of the his/her tweet has more significance compared to a tweet by a "less significant" tweeter, with less followers.

In [403]:
df_tweet_SA['Compound_multiplied'] = df_tweet_SA['compound']*df_tweet_SA['follower count']

In [404]:
df_tweet_SA.head()

,datetime,text,follower count,compound,neg,neu,pos,Compound_multiplied
1,2019-03-28,RT @247WallSt: Thursday’s Top Analyst Upgrades...,32,0.2023,0.000,0.917,0.083,6.4736
4,2019-03-28,Western Digital $WDC Holding Boosted by Orca I...,419,0.4939,0.048,0.801,0.151,206.9441
6,2019-03-28,Top Bullish Flow (a/o12:50pmEST): $BAC $FB $W ...,5475,0.5574,0.000,0.805,0.195,3051.7650
7,2019-03-28,$WDC - Western Digital: Deep Value Trap Or Gem...,16,0.0258,0.197,0.598,0.205,0.4128
9,2019-03-28,#estate asset watch - stocks with the highest ...,12567,0.6705,0.000,0.899,0.101,8426.1735


### Remove rows where 'follower count' is NaN

In [405]:
nan_rows = df_tweet_SA[df_tweet_SA['follower count'].isnull()]
nan_rows

,datetime,text,follower count,compound,neg,neu,pos,Compound_multiplied


In [406]:
df_tweet_SA = df_tweet_SA[np.isfinite(df_tweet_SA['follower count'])]

### Create a df with daily MEANS of each column 


In [407]:
df_daily_mean=(df_tweet_SA.groupby(df_tweet_SA.datetime).mean())

df_daily_mean.tail()

,follower count,compound,neg,neu,pos,Compound_multiplied
datetime,,,,,,
2019-03-24,1795.666667,0.301833,0.024000,0.857778,0.118333,664.875244
2019-03-25,9106.875000,0.501550,0.026937,0.769750,0.203312,1562.571862
2019-03-26,802.333333,0.319411,0.042222,0.836944,0.120833,84.944561
2019-03-27,1880.969697,0.168927,0.035091,0.861788,0.103121,659.380088
2019-03-28,10861.230769,0.400405,0.036846,0.764385,0.198718,3291.307072


In [408]:
len(df_daily_mean)

28

## Downloading stock data from Yahoo Finance - remember to adjust dates when needed
They should be the same as for the new, scraped twitter data 

In [409]:
#import pandas_datareader.data as web

start = dt.datetime(2019, 2, 28)
end =  dt.datetime(2019, 3, 28) #dt.datetime.now() 

df_stock = web.DataReader(stock, 'yahoo', start, end)

In [410]:
df_stock.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-03-22,51.450001,48.369999,51.250000,48.389999,8670800.0,47.856834
2019-03-25,48.849998,47.090000,48.250000,47.889999,4426500.0,47.362343
2019-03-26,48.930000,46.869999,48.189999,47.080002,6381900.0,46.561272
2019-03-27,47.450001,44.939999,47.230000,45.380001,8924400.0,44.880001
2019-03-28,46.279999,44.759998,44.919998,45.750000,5710600.0,45.750000


### New column for daily percent change - stock

In [411]:
df_stock['Pct_change'] = (df_stock['Close'] - df_stock['Open']) / df_stock['Open'] * 100.0

In [412]:
df_stock.tail()

,High,Low,Open,Close,Volume,Adj Close,Pct_change
Date,,,,,,,
2019-03-22,51.450001,48.369999,51.250000,48.389999,8670800.0,47.856834,-5.580489
2019-03-25,48.849998,47.090000,48.250000,47.889999,4426500.0,47.362343,-0.746115
2019-03-26,48.930000,46.869999,48.189999,47.080002,6381900.0,46.561272,-2.303376
2019-03-27,47.450001,44.939999,47.230000,45.380001,8924400.0,44.880001,-3.916999
2019-03-28,46.279999,44.759998,44.919998,45.750000,5710600.0,45.750000,1.847733


### Combine the tweet sentiment dataframe with the stock data dataframe

In [413]:
df_full = pd.concat([df_stock[['High', 'Low', 'Open', 'Adj Close', 'Pct_change']],\
                     df_daily_mean], axis=1, sort=False)
df_full.tail(11)

,High,Low,Open,Adj Close,Pct_change,follower count,compound,neg,neu,pos,Compound_multiplied
2019-03-18,48.669998,47.830002,48.189999,47.757938,0.207517,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-19,48.840000,47.720001,48.720001,47.352455,-1.724138,1701.100000,0.549080,0.042500,0.756800,0.200600,1048.109500
2019-03-20,48.169998,46.560001,47.840000,46.650276,-1.400506,1040.625000,0.308138,0.024250,0.849938,0.125750,343.374687
2019-03-21,52.270000,47.869999,47.869999,51.209480,8.167955,4087.935065,0.290555,0.036558,0.832312,0.131117,1253.919414
2019-03-22,51.450001,48.369999,51.250000,47.856834,-5.580489,3233.733333,0.276140,0.020133,0.875667,0.104200,1158.498953
2019-03-23,NaN,NaN,NaN,NaN,NaN,2556.142857,0.240871,0.026857,0.832000,0.141143,975.690900
2019-03-24,NaN,NaN,NaN,NaN,NaN,1795.666667,0.301833,0.024000,0.857778,0.118333,664.875244
2019-03-25,48.849998,47.090000,48.250000,47.362343,-0.746115,9106.875000,0.501550,0.026937,0.769750,0.203312,1562.571862
2019-03-26,48.930000,46.869999,48.189999,46.561272,-2.303376,802.333333,0.319411,0.042222,0.836944,0.120833,84.944561
2019-03-27,47.450001,44.939999,47.230000,44.880001,-3.916999,1880.969697,0.168927,0.035091,0.861788,0.103121,659.380088


### Impute missing data with their means
For some of the cashtag twitter data entire days were missing. Mean imputation replaces missing values with the mean value of that feature/variable. Mean imputation is one of the most ‘naive’ imputation methods because unlike more complex methods like k-nearest neighbors imputation, it does not use the information we have about an observation to estimate a value for it.

It's not pretty, it's not fully accurate, but better than removing these days altogether. Besides, there might be other missing days for the other cashtag tweets and this cell fixes them all.

In [414]:
df_full['follower count'].fillna(df_full['follower count'].mean(), inplace=True)
df_full['compound'].fillna(df_full['compound'].mean(), inplace=True)
df_full['neg'].fillna(df_full['neg'].mean(), inplace=True)
df_full['neu'].fillna(df_full['neu'].mean(), inplace=True)
df_full['pos'].fillna(df_full['pos'].mean(), inplace=True)
df_full['Compound_multiplied'].fillna(df_full['Compound_multiplied'].mean(), inplace=True)

df_full.head()

,High,Low,Open,Adj Close,Pct_change,follower count,compound,neg,neu,pos,Compound_multiplied
2019-02-27,51.369999,48.50,50.830002,48.005184,-4.505215,4943.724921,0.298596,0.037299,0.830733,0.131963,1111.553143
2019-02-28,50.520000,46.66,48.150002,49.745789,4.465208,916.600000,0.255800,0.023200,0.915200,0.061600,369.520540
2019-03-01,51.950001,50.59,51.020000,51.090805,1.254409,6370.678571,0.378829,0.035893,0.805286,0.158893,2136.179221
2019-03-02,NaN,NaN,NaN,NaN,NaN,1557.916667,0.535575,0.021583,0.785583,0.192750,886.068175
2019-03-03,NaN,NaN,NaN,NaN,NaN,5174.818182,0.296073,0.010545,0.883000,0.106455,1072.697936


### Interpolate for missing weekend stock data!
NOTE! Last date has no values (=NaN) for 'Predicted_change_stock' or 'Buy/Sell'.
NOTE also! Tweets were missing for day Monday 18.3.2019 so sentiment values interpolated also for that date... :-(

In [415]:
df_full = df_full[[ 'High', 'Low', 'Open', 'Adj Close', 'Pct_change', 'follower count', 'compound','neg', 'neu', 'pos',\
         'Compound_multiplied' ]].interpolate(method='linear', limit_direction='forward', axis=0)


In [416]:
df_full.tail(22)

,High,Low,Open,Adj Close,Pct_change,follower count,compound,neg,neu,pos,Compound_multiplied
2019-03-07,47.860001,45.410000,47.860001,46.472260,-1.817800,2621.500000,0.307390,0.045350,0.823650,0.130950,1406.704325
2019-03-08,47.560001,45.730000,46.040001,46.956860,3.127712,13547.103448,0.151948,0.048069,0.853172,0.098759,-2245.066659
2019-03-09,48.373334,46.373333,46.610001,47.675521,3.418475,702.916667,0.360208,0.036833,0.799750,0.163417,-34.028050
2019-03-10,49.186667,47.016666,47.180000,48.394182,3.709237,2524.800000,-0.288960,0.091500,0.887200,0.021300,34.763340
2019-03-11,50.000000,47.660000,47.750000,49.112843,4.000000,3704.083333,0.399446,0.024542,0.785167,0.190292,1394.352987
2019-03-12,50.560001,48.750000,49.689999,48.608463,-1.086732,4934.961538,0.183062,0.045692,0.852692,0.101615,648.260250
2019-03-13,49.470001,48.070000,49.139999,48.450226,-0.305246,2240.200000,0.235567,0.051667,0.796400,0.152000,744.901227
2019-03-14,48.720001,47.150002,48.599998,47.055759,-2.098759,2547.928571,0.392025,0.012607,0.835821,0.151571,1281.700721
2019-03-15,49.130001,47.639999,47.650002,47.570026,0.944380,1986.500000,0.067900,0.064111,0.864333,0.071556,520.731389
2019-03-16,48.976667,47.703334,47.830001,47.632664,0.698759,710.428571,0.595800,0.017286,0.853286,0.129429,502.585871


In [417]:
pd.DataFrame.describe(df_full)

,High,Low,Open,Adj Close,Pct_change,follower count,compound,neg,neu,pos,Compound_multiplied
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,49.835334,47.827333,48.874667,48.266263,-0.127772,4943.724921,0.298596,0.037299,0.830733,0.131963,1111.553143
std,1.785783,1.738155,1.855279,1.800938,3.106129,6015.218435,0.176918,0.020698,0.038520,0.042410,1824.947502
min,46.279999,44.759998,44.919998,44.880001,-5.580489,527.625000,-0.288960,0.010545,0.756800,0.021300,-2245.066659
25%,48.745834,46.906666,47.832500,47.354927,-2.252222,1724.741667,0.236893,0.024063,0.798906,0.104764,402.786873
50%,49.328334,47.711667,48.219999,47.693652,-0.048864,2588.821429,0.300214,0.036226,0.833864,0.131033,930.879538
75%,51.173333,48.302499,50.332500,48.986748,1.204081,5117.044867,0.395710,0.044637,0.853257,0.157170,1274.755395
max,53.400002,51.580002,52.270000,52.030338,8.167955,30960.096774,0.595800,0.100806,0.915200,0.203312,7489.434700


### Create  'label' -column for the forecast; 'Predicted_change' for the next day - ADJUST FOR THE NR OF DAYS!
With forecasting, you want to forecast out a certain number of datapoints. Let's we want to forecast out 1% of the entire length of the dataset. Thus, if our data is 100 days of stock prices and you are just trying to predict tomorrow's price, then you would just do 1 day out, and the forecast would be just one day out. If you predict 10 days out, you can actually generate a forecast for every day, for the next week and a half.

In this case, with 30 days of data, one day would reprecent 3.33 % of the data

I just add a new column - 'Predicted_change' - into the dataframe with a simple pandas operation:

Previous day's 'Compound_multiplied_scaled' should predict next day's 'Pct_change'.

In [418]:
len(df_full)

30

In [419]:
#import math

forecast_col = 'Pct_change'

forecast_out = int(math.ceil(0.0333 * len(df_full)))

df_full['Predicted_change'] = df_full[forecast_col].shift(-forecast_out)

### Create another 'label' -column - 'Buy/Sell' - which is 1 if 'Predicted_change' is positive (=buy) and -1 if negative (=sell)

In [420]:
buy_or_sell = []

for row in df_full['Pct_change']:
    if row >= 0:
        buy_or_sell.append(1)
    elif row < 0:
        buy_or_sell.append(-1) 

#Adds minus 1 or 1 to the column based on if 'Predicted_change' is negative or positive
df_full['Buy/Sell'] = buy_or_sell

# The 'Buy/Sell' values need to be shifte up on row to match the 'Predicted_change' values
df_full['Buy/Sell'] = df_full['Buy/Sell'].shift(-1)

In [421]:
df_full.head(50)

,High,Low,Open,Adj Close,Pct_change,follower count,compound,neg,neu,pos,Compound_multiplied,Predicted_change,Buy/Sell
2019-02-27,51.369999,48.500000,50.830002,48.005184,-4.505215,4943.724921,0.298596,0.037299,0.830733,0.131963,1111.553143,4.465208,1.0
2019-02-28,50.520000,46.660000,48.150002,49.745789,4.465208,916.600000,0.255800,0.023200,0.915200,0.061600,369.520540,1.254409,1.0
2019-03-01,51.950001,50.590000,51.020000,51.090805,1.254409,6370.678571,0.378829,0.035893,0.805286,0.158893,2136.179221,1.053096,1.0
2019-03-02,52.433334,50.920001,51.436667,51.403983,1.053096,1557.916667,0.535575,0.021583,0.785583,0.192750,886.068175,0.851782,1.0
2019-03-03,52.916668,51.250001,51.853334,51.717161,0.851782,5174.818182,0.296073,0.010545,0.883000,0.106455,1072.697936,0.650469,1.0
2019-03-04,53.400002,51.580002,52.270000,52.030338,0.650469,10528.862069,0.396938,0.011172,0.851897,0.136931,7489.434700,-2.423074,-1.0
2019-03-05,52.240002,50.480000,52.000000,50.180943,-2.423074,9800.666667,0.139700,0.055250,0.835417,0.109333,-1935.955642,-4.447978,-1.0
2019-03-06,50.360001,48.099998,50.360001,47.589809,-4.447978,30960.096774,0.048803,0.100806,0.796677,0.102484,5775.218777,-1.817800,-1.0
2019-03-07,47.860001,45.410000,47.860001,46.472260,-1.817800,2621.500000,0.307390,0.045350,0.823650,0.130950,1406.704325,3.127712,1.0
2019-03-08,47.560001,45.730000,46.040001,46.956860,3.127712,13547.103448,0.151948,0.048069,0.853172,0.098759,-2245.066659,3.418475,1.0


### Save 'df_full' as excel file with the $cashtag as file name
The files created with this notebook are used when testing how well the binary classifiers trained on data from 2016 predict stock price movements today, i.e. during March 2019!

### Change the name of the excel-file that will be created below!

In [422]:
# import xlsxwriter
# import os

writer_df = pd.ExcelWriter('df_full.xlsx', engine='xlsxwriter')
df_full.to_excel(writer_df)
writer_df.save()

os.rename('df_full.xlsx', '$WDC_prediction.xlsx') # UPDATE THE $CASHTAG BEFORE RUNNING THE CELL!!!!